Calcular el human-participants-agreement

Tomo el conjunto de tweets en el que etiquetamos en común

In [1]:
import pandas as pd
from nltk import agreement

In [4]:
# English texts agreement
df = pd.read_csv('toLabel//proc//en//MNBall-predictions-en.csv', sep='\t', encoding='utf-8')
rater1 = df['EmotionEN-Axel']
rater2 = df['EmotionEN-Ana']
rater3 = df['EmotionEN-Fernando']
rater4 = df['EmotionEN-Mauro']

taskdata=[[0,str(i),str(rater1[i])] for i in range(0,len(rater1))]+[[1,str(i),str(rater2[i])] for i in range(0,len(rater2))]+[[2,str(i),str(rater3[i])] for i in range(0,len(rater3))] + [[0,str(i),str(rater4[i])] for i in range(0,len(rater4))]
ratingtask = agreement.AnnotationTask(data=taskdata)
print("kappa " +str(ratingtask.kappa()))
print("fleiss " + str(ratingtask.multi_kappa()))
print("alpha " +str(ratingtask.alpha()))
print("scotts " + str(ratingtask.pi()))

kappa 0.23262517615407033
fleiss 0.22883787661406027
alpha 0.34395209580838326
scotts 0.1886792452830189


In [8]:
# Spanish texts agreement
df = pd.read_csv('toLabel//proc//es//MNBall-predictions-es.csv', sep='\t', encoding='utf-8')
rater1 = df['EmotionES-Axel']
rater2 = df['EmotionES-Ana']
rater3 = df['EmotionES-Fernando']
rater4 = df['EmotionES-Mauro']

taskdata=[[0,str(i),str(rater1[i])] for i in range(0,len(rater1))]+[[1,str(i),str(rater2[i])] for i in range(0,len(rater2))]+[[2,str(i),str(rater3[i])] for i in range(0,len(rater3))] + [[0,str(i),str(rater4[i])] for i in range(0,len(rater4))]
ratingtask = agreement.AnnotationTask(data=taskdata)
print("kappa " +str(ratingtask.kappa()))
print("fleiss " + str(ratingtask.multi_kappa()))
print("alpha " +str(ratingtask.alpha()))
print("scotts " + str(ratingtask.pi()))

kappa 0.1619228139131934
fleiss 0.16302765647743817
alpha 0.31482375663930473
scotts 0.11742133537989263


### Calcular la precision de la máquina al predecir los tweets que etiquetamos por separado

Recordar cambiar todos los 'new' (de los archivos nuevos de prueba) por 'proc'

In [5]:
emotions = ["No emotion", "Joy", "Anger", "Sadness", "Disgust", "Fear", "Trust", "Surprise"]
names = ['Axel', 'Ana', 'Fernando', 'Mauro']
lang = 'en'

In [35]:
#df = pd.read_csv('toLabel//proc//' + lang + '//MNBall-predictions-' + lang + '.csv', sep='\t', encoding='utf-8')
df = pd.read_csv('toLabel//new//'+lang+'//Plutchick-DS0-RF-all-predictions-'+lang+'.csv', sep='\t', encoding='utf-8')

In [36]:
df.head()

,text,EmotionEN-Axel,EmotionEN-Ana,EmotionEN-Fernando,EmotionEN-Mauro,EmotionEN-Machine
0,@realDonaldTrump Good thing you shut down the ...,Anger,Anger,Anger,Anger,No emotion
1,@realDonaldTrump Energy jobs include ALL forms...,Anger,Anger,Disgust,Disgust,Joy
2,@realDonaldTrump We don't want the Keystone Pi...,Anger,Anger,Trust,Anger,Joy
3,@WhiteHouse No more bills that do not include ...,Fear,Anger,No emotion,No emotion,No emotion
4,@WhiteHouse @realDonaldTrump Good to hear food...,Joy,Joy,Joy,Joy,No emotion


Construyo una columna con la emocion con mas votos humanos

In [37]:
# Put all votes in a list of lists
votes = list()

for i in range(len(df)):
    votesRow = [0, 0, 0, 0, 0, 0, 0, 0]
    for name in names:
        emotion = df.iloc[i]['Emotion' + lang.upper() + '-' + name]
        index = emotions.index(emotion)
        votesRow[index] += 1
    votes.append(votesRow)
print(votes)

[[0, 0, 4, 0, 0, 0, 0, 0], [0, 0, 2, 0, 2, 0, 0, 0], [0, 0, 3, 0, 0, 0, 1, 0], [2, 0, 1, 0, 0, 1, 0, 0], [0, 4, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 3, 0], [3, 0, 0, 0, 0, 0, 1, 0], [3, 0, 1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 1, 1, 0], [2, 0, 1, 0, 0, 0, 1, 0], [0, 0, 1, 0, 3, 0, 0, 0], [0, 0, 0, 2, 0, 2, 0, 0], [0, 0, 3, 0, 1, 0, 0, 0], [0, 0, 2, 1, 0, 1, 0, 0], [3, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 3, 0, 0, 0, 0], [0, 0, 0, 3, 1, 0, 0, 0], [2, 0, 0, 0, 0, 0, 2, 0], [0, 0, 0, 1, 0, 3, 0, 0], [3, 0, 0, 0, 0, 0, 1, 0], [2, 0, 0, 0, 0, 2, 0, 0], [0, 4, 0, 0, 0, 0, 0, 0], [0, 2, 0, 0, 0, 0, 2, 0], [1, 1, 1, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 3, 0]]


In [38]:
# Decide what emotion is the selected one based on the votes
# If there's a tie, all emotions in the tie are valid.
# These are put in the column separated by a '-'
votesEmotions = list()

for listVotes in votes:
    mostVotes = list()
    maxVotes = 0
    # Search which is the highest score
    for v in listVotes:
        if (v > maxVotes):
            maxVotes = v
            
    # Add the most voted emotions
    
    for i in range(len(listVotes)):
        if (listVotes[i] == maxVotes):
            mostVotes.append(emotions[i])
    votesEmotions.append(mostVotes)
    
# If two emotions have the highest amount of votes, add an empty string
for i in range(len(votesEmotions)):
    if (len(votesEmotions[i]) > 1):
        votesEmotions[i] = [""]
    
print(votesEmotions)

[['Anger'], [''], ['Anger'], ['No emotion'], ['Joy'], ['Trust'], ['No emotion'], ['No emotion'], [''], ['No emotion'], ['Disgust'], [''], ['Anger'], ['Anger'], ['No emotion'], ['Sadness'], ['Sadness'], [''], ['Fear'], ['No emotion'], [''], ['Joy'], [''], [''], ['Trust']]


In [39]:
# Construct the column with the most voted emotions
dicEmotions = dict()
dicEmotions['Emotion'+lang.upper()+'-Humans'] = list()

for i in range(len(votesEmotions)):
    strEmotions = '-'.join(votesEmotions[i])
    dicEmotions['Emotion'+lang.upper()+'-Humans'].append(strEmotions)

df['Emotion'+lang.upper()+'-Humans'] = dicEmotions['Emotion'+lang.upper()+'-Humans']

In [41]:
#df = df.drop(labels = 'Unnamed: 0', axis=1)
#df.to_csv('toLabel//proc//' + lang + '//MNBall-predictions-' + lang + '.csv', sep='\t', encoding='utf-8', index=False)
df.to_csv('toLabel//new//'+lang+'//Plutchick-DS0-RF-all-predictions-'+lang+'.csv', sep='\t', encoding='utf-8', index=False)

In [42]:
df.head()

,text,EmotionEN-Axel,EmotionEN-Ana,EmotionEN-Fernando,EmotionEN-Mauro,EmotionEN-Machine,EmotionEN-Humans
0,@realDonaldTrump Good thing you shut down the ...,Anger,Anger,Anger,Anger,No emotion,Anger
1,@realDonaldTrump Energy jobs include ALL forms...,Anger,Anger,Disgust,Disgust,Joy,
2,@realDonaldTrump We don't want the Keystone Pi...,Anger,Anger,Trust,Anger,Joy,Anger
3,@WhiteHouse No more bills that do not include ...,Fear,Anger,No emotion,No emotion,No emotion,No emotion
4,@WhiteHouse @realDonaldTrump Good to hear food...,Joy,Joy,Joy,Joy,No emotion,Joy


Ahora siiiiiiiiiiiiiiiiiiiiiiiiii

Calculo la precision de la maquina

In [43]:
for lang in ['es', 'en']:
    #df = pd.read_csv('toLabel//proc//' + lang + '//MNBall-predictions-' + lang + '.csv', sep='\t', encoding='utf-8').dropna(axis=0,how='any')
    df = pd.read_csv('toLabel//new//'+lang+'//Plutchick-DS0-RF-all-predictions-'+lang+'.csv', sep='\t', encoding='utf-8')
    df = df.reset_index()
    machine = df['Emotion'+lang.upper()+'-Machine']
    human = df['Emotion'+lang.upper()+'-Humans']
    hits = 0
    
    for i in range(len(machine)):
        if (machine[i] == human[i]):
            hits = hits + 1
    print("Machine accuracy: " + str(lang) + "\n hits/tweets: " + str(hits) + "/" + str(len(machine)) + "\n" + str(hits / len(machine)))

Machine accuracy: es
 hits/tweets: 4/25
0.16
Machine accuracy: en
 hits/tweets: 6/25
0.24


# Lo mismo pero con POSNEGNEU

In [51]:
import numpy as np
lang = 'en'
#df = pd.read_csv('toLabel//proc//' + lang + '//MNBall-predictions-' + lang + '.csv', sep='\t', encoding='utf-8')
df = pd.read_csv('toLabel//new//'+lang+'//Plutchick-DS0-RF-all-predictions-'+lang+'.csv', sep='\t', encoding='utf-8')
df = df.replace(np.nan, '', regex=True)
df.head()

,text,EmotionEN-Axel,EmotionEN-Ana,EmotionEN-Fernando,EmotionEN-Mauro,EmotionEN-Machine,EmotionEN-Humans
0,@realDonaldTrump Good thing you shut down the ...,Anger,Anger,Anger,Anger,No emotion,Anger
1,@realDonaldTrump Energy jobs include ALL forms...,Anger,Anger,Disgust,Disgust,Joy,
2,@realDonaldTrump We don't want the Keystone Pi...,Anger,Anger,Trust,Anger,Joy,Anger
3,@WhiteHouse No more bills that do not include ...,Fear,Anger,No emotion,No emotion,No emotion,No emotion
4,@WhiteHouse @realDonaldTrump Good to hear food...,Joy,Joy,Joy,Joy,No emotion,Joy


Emociones originales:

0. No_emo
1. Joy
2. Anger
3. Sadness
4. Disgust
5. Fear
6. Trust
7. Surprise
8. Anticipation


Los sentimientos los clasifico en:
+ Positivo: [Joy, Trust]
+ Negativo: [Anger, Sadness, Fear, Disgust]
+ Neutro: [Surprise, Anticipation, No emo]


In [52]:
pnnDic = {'No emotion':'Neu', 'Joy':'Pos', 'Anger':'Neg', 'Sadness':'Neg', 'Disgust':'Neg', 
          'Fear':'Neg', 'Trust':'Pos', 'Surprise':'Neu', 'Anticipation':'Neu', '':''}

In [53]:
pnnList = list()

for i in range(len(df['Emotion'+lang.upper()+'-Humans'])):
    emotion = df.iloc[i]['Emotion'+lang.upper()+'-Humans']
    pnn = pnnDic[emotion]
    pnnList.append(pnn)
    
df['PNN'+lang.upper()+'-Humans'] = pnnList
#df.to_csv('toLabel//proc//' + lang + '//MNBall-predictions-' + lang + '.csv', sep='\t', encoding='utf-8', index=False)
df.to_csv('toLabel//new//'+lang+'//Plutchick-DS0-RF-all-predictions-'+lang+'.csv', sep='\t', encoding='utf-8', index=False)

In [54]:
df.head()

,text,EmotionEN-Axel,EmotionEN-Ana,EmotionEN-Fernando,EmotionEN-Mauro,EmotionEN-Machine,EmotionEN-Humans,PNNEN-Humans
0,@realDonaldTrump Good thing you shut down the ...,Anger,Anger,Anger,Anger,No emotion,Anger,Neg
1,@realDonaldTrump Energy jobs include ALL forms...,Anger,Anger,Disgust,Disgust,Joy,,
2,@realDonaldTrump We don't want the Keystone Pi...,Anger,Anger,Trust,Anger,Joy,Anger,Neg
3,@WhiteHouse No more bills that do not include ...,Fear,Anger,No emotion,No emotion,No emotion,No emotion,Neu
4,@WhiteHouse @realDonaldTrump Good to hear food...,Joy,Joy,Joy,Joy,No emotion,Joy,Pos


In [55]:
pnnList = list()

for i in range(len(df['Emotion'+lang.upper()+'-Machine'])):
    emotion = df.iloc[i]['Emotion'+lang.upper()+'-Machine']
    pnn = pnnDic[emotion]
    pnnList.append(pnn)
    
df['PNN'+lang.upper()+'-Machine'] = pnnList
#df.to_csv('toLabel//proc//' + lang + '//MNBall-predictions-' + lang + '.csv', sep='\t', encoding='utf-8', index=False)
df.to_csv('toLabel//new//'+lang+'//Plutchick-DS0-RF-all-predictions-'+lang+'.csv', sep='\t', encoding='utf-8', index=False)

Calculo el agreement entre los humanos y la maquina en base a posnegneu

In [56]:
df.head()

,text,EmotionEN-Axel,EmotionEN-Ana,EmotionEN-Fernando,EmotionEN-Mauro,EmotionEN-Machine,EmotionEN-Humans,PNNEN-Humans,PNNEN-Machine
0,@realDonaldTrump Good thing you shut down the ...,Anger,Anger,Anger,Anger,No emotion,Anger,Neg,Neu
1,@realDonaldTrump Energy jobs include ALL forms...,Anger,Anger,Disgust,Disgust,Joy,,,Pos
2,@realDonaldTrump We don't want the Keystone Pi...,Anger,Anger,Trust,Anger,Joy,Anger,Neg,Pos
3,@WhiteHouse No more bills that do not include ...,Fear,Anger,No emotion,No emotion,No emotion,No emotion,Neu,Neu
4,@WhiteHouse @realDonaldTrump Good to hear food...,Joy,Joy,Joy,Joy,No emotion,Joy,Pos,Neu


In [57]:
for lang in ['es', 'en']:
    #df = pd.read_csv('toLabel//proc//' + lang + '//MNBall-predictions-' + lang + '.csv', sep='\t', encoding='utf-8').dropna(axis=0,how='any')
    df = pd.read_csv('toLabel//new//'+lang+'//Plutchick-DS0-RF-all-predictions-'+lang+'.csv', sep='\t', encoding='utf-8')
    df = df.reset_index()
    human = df['PNN'+lang.upper()+'-Humans']
    machine = df['PNN'+lang.upper()+'-Machine']
    hits = 0

    for i in range(len(machine)):
        if (machine[i] == human[i]):
            hits = hits + 1
    print("Machine accuracy: " + str(lang) + "\n hits/tweets: " + str(hits) + "/" + str(len(machine)) + "\n" + str(hits / len(machine)))

Machine accuracy: es
 hits/tweets: 5/25
0.2
Machine accuracy: en
 hits/tweets: 8/25
0.32


Calculo la precision de cada uno de los participantes en el proceso de etiquetado sobre el modelo de plutchick

In [9]:
for lang in ['es', 'en']:
    #df = pd.read_csv('toLabel//all-predictions-' + lang + '.csv', sep='\t', encoding='utf-8').dropna(axis=0, how='any')
    df = pd.read_csv('toLabel//new//'+lang+'//Plutchick-DS0-RF-all-predictions-'+lang+'.csv', sep='\t', encoding='utf-8')
    df = df.reset_index()
    print(lang)
    
    for name in names:
        rater = df['Emotion' + lang.upper() + '-' + name]
        humans = df['Emotion'+lang.upper()+'-Humans']
        hits = 0
        
        for i in range(len(rater)):
            if (rater[i] == humans[i]):
                hits = hits + 1
        print(name + " accuracy: \n hits/tweets: " + str(hits) + "/" + str(len(humans)) + "\n" + str(hits / len(humans)))
        print("\n")

es
Axel accuracy: 
 hits/tweets: 15/25
0.6


Ana accuracy: 
 hits/tweets: 15/25
0.6


Fernando accuracy: 
 hits/tweets: 10/25
0.4


Mauro accuracy: 
 hits/tweets: 15/25
0.6


en
Axel accuracy: 
 hits/tweets: 15/25
0.6


Ana accuracy: 
 hits/tweets: 9/25
0.36


Fernando accuracy: 
 hits/tweets: 16/25
0.64


Mauro accuracy: 
 hits/tweets: 14/25
0.56




Calculo la precision de cada uno de los participantes en el proceso de etiquetado sobre el modelo de POSNEGNEU

In [165]:
for lang in ['es', 'en']:
    #df = pd.read_csv('toLabel//all-predictions-' + lang + '.csv', sep='\t', encoding='utf-8').dropna(axis=0, how='any')
    df = pd.read_csv('toLabel//new//'+lang+'//Plutchick-DS0-RF-all-predictions-'+lang+'.csv', sep='\t', encoding='utf-8')
    df = df.reset_index()
    print(lang)
    
    for name in names:
        rater = df['Emotion' + lang.upper() + '-' + name]
        humans = df['Emotion'+lang.upper()+'-Humans']
        hits = 0
        
        for i in range(len(rater)):
            if (pnnDic[rater[i]] == pnnDic[humans[i]]):
                hits = hits + 1
        print(name + " accuracy: \n hits/tweets: " + str(hits) + "/" + str(len(machine)) + "\n" + str(hits / len(machine)))
        print("\n")

es
Axel accuracy: 
 hits/tweets: 17/18
0.9444444444444444


Ana accuracy: 
 hits/tweets: 16/18
0.8888888888888888


Fernando accuracy: 
 hits/tweets: 15/18
0.8333333333333334


Mauro accuracy: 
 hits/tweets: 17/18
0.9444444444444444


en
Axel accuracy: 
 hits/tweets: 16/18
0.8888888888888888


Ana accuracy: 
 hits/tweets: 11/18
0.6111111111111112


Fernando accuracy: 
 hits/tweets: 17/18
0.9444444444444444


Mauro accuracy: 
 hits/tweets: 16/18
0.8888888888888888


